# 흑백이미지 복원
## ALGORITHMIA의 Colorize Photos
* On-line App : https://demos.algorithmia.com/colorize-photos
* The project is also available at GitHub here -> https://github.com/shubham0204/ImageColorization






# 1) Importing the libraries
We import [TensorFlow](https://www.tensorflow.org/), [Numpy](http://www.numpy.org/) and [scikit-image](https://scikit-image.org/) module for playing around with images. Printing out the TensorFlow version is a good practice to see that everything is working fine. ( See [Colorizer.py](https://github.com/shubham0204/ImageColorization/blob/master/Colorizer.py) on GitHub )

In [ ]:
%tensorflow_version 1.x
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import models
from skimage.io import imshow , imsave
import tensorflow as tf
import numpy as np

print( tf.VERSION ) 


In [ ]:
%cd /content/
%rm -rf ImageColorization
!git clone https://github.com/shubham0204/ImageColorization
%cd /content/ImageColorization

# 2) Loading the data
We load the data which is stored in .npy files using the `np.load` method. We print out the shapes of the training and testing data.


*   To convert your own images to .npy files, refer to the GitHub project and see the [Parser.py ](https://github.com/shubham0204/ImageColorization/blob/master/Parser.py)file.
*   For sample data, download the three files from the [sample_data directory](https://github.com/shubham0204/ImageColorization/tree/master/sample_data) of the GitHub project.




In [ ]:
!find .|grep model

In [ ]:

X = np.load( './sample_data/X.npy' ) 
Y = np.load( './sample_data/Y.npy' ) 
test_X = np.load( './sample_data/test_X.npy' ) 

print( X.shape ,Y.shape ) 
print( test_X.shape ) 



In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X[0].reshape( (64,64))  ,cmap='gray')


In [ ]:
plt.imshow(Y[0])


# 3) Defining the model
We define our Convolutional Auto Encoder model using the Keras [Sequential API](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential).


*   We use a low dropout rate of 0.3,
*   `DIMEN` is the dimension of the image which acts as an input to the model. In our case, it is 64 or the image has dimensions of 64px * 64px.

We compile the model to use the Adam optimizer with a learning rate of 0.0001. The loss function is mean squared error and we output the `mae` or mean absolute error of the model during training. ( See [Colorizer.py](https://github.com/shubham0204/ImageColorization/blob/master/Colorizer.py) on GitHub )



In [ ]:

dropout_rate = 0.5
DIMEN = 64
kernel_size = ( 4 , 4 )

NEURAL_SCHEMA = [

    Conv2D( 32 , input_shape=( DIMEN , DIMEN , 1 ) , kernel_size=kernel_size , strides=1,activation=relu),
    Dropout( dropout_rate ) ,
    Conv2D( 64, kernel_size=kernel_size, strides=1, activation=relu),
    Dropout(dropout_rate),
    Conv2D( 128, kernel_size=kernel_size, strides=1, activation=relu) ,
    Dropout(dropout_rate),
    Conv2D( 256, kernel_size=kernel_size, strides=1, activation=relu),
    Dropout(dropout_rate),
    Conv2DTranspose( 128, kernel_size=kernel_size, strides=1, activation=relu),
    Dropout(dropout_rate),
    Conv2DTranspose( 64, kernel_size=kernel_size, strides=1, activation=relu),
    Dropout(dropout_rate),
    Conv2DTranspose( 32, kernel_size=kernel_size, strides=1, activation=relu),
    Dropout(dropout_rate),
    Conv2DTranspose( 3, kernel_size=kernel_size, strides=1, activation=tanh ),

]

model = tf.keras.Sequential( NEURAL_SCHEMA )

model.compile(
    optimizer=optimizers.Adam(0.0001),
    loss=losses.mean_squared_error,
    metrics=['mae'],
)


# 4) Training the model
We train the model over the training dataset using the `fit()` method.  ( See [Colorizer.py](https://github.com/shubham0204/ImageColorization/blob/master/Colorizer.py) on GitHub )


**Note : If required, load the model which was trained earlier or use the model located in the [models](https://github.com/shubham0204/ImageColorization/tree/master/models) directory on GitHub.**

In [ ]:

model = models.load_model( './models/final_model.h5' ) 


In [ ]:

model.fit(
    X, 
    Y, 
    batch_size=3 , 
    epochs=500
)


We save the model to export it or download it to our local machine using `model.save()` method. **Tip : To use it on an Android or iOS device, use [this notebook](https://colab.research.google.com/drive/1IUIn9ffk5ICKujqPyuGaHL2irQ9Wmtpm) to convert the model.h5 to a [TensorFlow Lite](https://www.tensorflow.org/lite) model ( .tflite ).**

In [ ]:

model.save( 'model.h5')


# 5) Colorizing the images



1.   We take the grayscale images and feed them to our model
2.   To avoid negative values, we perform a operation to convert all negative values to 0.
3.   We multiply the values by 255 ( the images used for training were normalised ).
4.   Convert the values to a image array using `imsave( )` method.

( See [MainFile.py ](https://github.com/shubham0204/ImageColorization/blob/master/MainFile.py)on GitHub )





In [ ]:

values = model.predict( test_X )
values = np.maximum( values , 0 )

for i in range( 6 ):
    image_final = ( values[i] * 255).astype( np.uint8 )
    imsave( '{}.png'.format( i + 1 ) , image_final  )
    imshow( image_final )
